# Segmentación Simple por Color
## Ejemplo introductorio para procesamiento de imágenes

In [ ]:
# Bibliotecas necesarias
import numpy as np
from cv2 import imread
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

In [ ]:
# Función para analizar características de la imagen
def info_img(img):
    print('tamaño =', img.shape)
    print('máximo =', np.max(img))
    print('mínimo =', np.min(img))

In [ ]:
# Descarga y lectura de imagen
!wget https://plus.unsplash.com/premium_photo-1689245691840-2ab8b3bf9cd2?q=80&w=2114&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D

In [ ]:
img_cv = imread('/content/premium_photo-1689245691840-2ab8b3bf9cd2?q=80')      # formato opencv (BGR)
cv2_imshow(img_cv)
info_img(img_cv)

In [ ]:
# Conversión BGR a RGB
img = img_cv[:,:,[2,1,0]]          # BGR a RGB
plt.imshow(img)
plt.show()

In [ ]:
# Separación de canales de color
rojo = img[:,:,0]
verde = img[:,:,1]
azul = img[:,:,2]
canales = np.concatenate((rojo,verde,azul),axis=1)
cv2_imshow(canales)

In [ ]:
# Conversión a escala de grises
rojo_f = rojo.astype(float)
verde_f = verde.astype(float)
azul_f = azul.astype(float)
# Promedio ponderado
peso = (1/3,1/3,1/3)
#peso = (0.1,0.4,0.5)  # pesos alternativos
gris_f = peso[0]*rojo_f + peso[1]*verde_f + peso[2]*azul_f
gris = gris_f.astype(int)
info_img(gris)
cv2_imshow(gris)

In [ ]:
# Función para mostrar histograma
def histograma(X):
    (N,M) = X.shape
    n = 256
    h = np.zeros((256,))
    for i in range(N):
        for j in range(M):
            x = X[i,j]
            h[x] = h[x]+1
    plt.plot(range(n),h[0:n])
    plt.title('Histograma')
    plt.xlabel('Valor de pixel')
    plt.ylabel('Frecuencia')
    plt.show()

In [ ]:
histograma(gris)

In [ ]:
# Ajustar los rangos de color para incluir la flor rosa y el tallo verde
seg_r = (rojo > 100) & (rojo < 255)  # Rango de rojo más amplio para la flor rosa
seg_v = (verde > 0) & (verde < 150)  # Rango de verde más amplio para el tallo
seg_a = (azul > 0) & (azul < 150)  # Rango de azul más amplio para la flor rosa

# Combinar las condiciones para obtener la máscara de segmentación
seg_rgb = np.concatenate((seg_r, seg_v, seg_a), axis=1)
cv2_imshow(seg_rgb * 255)

In [ ]:
# Aplicar la operación lógica AND para obtener la segmentación final
seg = np.logical_and(seg_r, seg_v, seg_a)  # Incluir los tres canales
cv2_imshow(seg * 255)

In [ ]:
# Eliminación de píxeles aislados
(N,M) = seg.shape
limpia = seg.copy()
for i in range(N):
    s = np.sum(seg[i,:])
    if s<10:
        limpia[i,:] = 0
cv2_imshow(limpia*255)

In [ ]:
# Búsqueda de coordenadas del rectángulo
i_min = 1000
i_max = 0
j_min = 1000
j_max = 0
for i in range(N):
    for j in range(M):
        if limpia[i,j]>0:
            if i<i_min: i_min = i
            if i>i_max: i_max = i
            if j<j_min: j_min = j
            if j>j_max: j_max = j

In [ ]:
# Dibujo del rectángulo
y = [i_min,i_min,i_max,i_max,i_min]
x = [j_min,j_max,j_max,j_min,j_min]
plt.imshow(img)
plt.plot(x,y)
plt.title('Rectángulo delimitador')
plt.show()

In [ ]:
# Detección de bordes
borde = np.zeros((N,M),np.uint8)
# Bordes horizontales
for i in range(N):
    for j in range(1,M):
        if limpia[i,j]!=limpia[i,j-1]:
            borde[i,j] = 1
            borde[i,j-1] = 1
# Bordes verticales
for i in range(1,N):
    for j in range(M):
        if limpia[i-1,j]!=limpia[i,j]:
            borde[i,j] = 1
            borde[i,j-1] = 1
cv2_imshow(borde*255)

In [ ]:
# Resultado final con bordes en rojo
img_final = img_cv.copy()
for i in range(N):
    for j in range(M):
        if borde[i,j]==1:
            img_final[i,j,:] = [255,0,0]
cv2_imshow(img_final)